# <center> Lab 5-1: SocketCAN Broadcast Manager
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" />
### <center> Instructor: Dr. Jeremy Daily<br>Fall 2020

## Prerequisites
This notebook must be run on Linux with SocketCAN installed and running. If you can login to the command prompt, check for SocketCAN using `lsmod` as in the following:
```
debian@beaglebone:~$ lsmod | grep can
can_bcm                24576  1
can_raw                20480  0
vcan                   16384  0
```
This shows three CAN based kernel modules: one for BroadCast Manager (BCM), one for single CAN frames (RAW), and a virtual CAN module (vcan).

This notebook should be run with a jupyter server on the Linux machine. For example, if using the BeagleBone, start the server with the following command:
```
jupyter notebook --ip 192.168.7.2 --no-browser &
```
The `&` symbol runs the notebook server in the background.


# Using the Broadcast Manager (BCM)
The broadcast manager in SocketCAN is helpful in setting up periodic messages. It takes away the timing and loop requirements for the programmer and relegates it to the kernel module. This enables a programmer to focus on the sensing and simulation parts of the program instead of the messaging. 

This example goes through transmitting a wheel speed message using the BCM.

Details and Reference:

https://github.com/nebaruzdin/can-doc/blob/master/can-bcm.txt


## Create a BCM Socket
First, load the modules. Then, let's define the socket. BCM sockets are connected to, instead of being bound. 

We'll also define the data structures for the BCM header and the CAN Frame. From https://www.kernel.org/doc/Documentation/networking/can.txt:
```
   struct bcm_msg_head {
            __u32 opcode;                   /* command */
            __u32 flags;                    /* special flags */
            __u32 count;                    /* run 'count' times with ival1 */
            struct timeval ival1, ival2;    /* count and subsequent interval */
            canid_t can_id;                 /* unique can_id for task */
            __u32 nframes;                  /* number of can_frames following */
            struct can_frame frames[0];
    };
 ```
 and
 ```
   struct can_frame {
            canid_t can_id;  /* 32 bit CAN_ID + EFF/RTR/ERR flags */
            __u8    can_dlc; /* frame payload length in byte (0 .. 8) */
            __u8    __pad;   /* padding */
            __u8    __res0;  /* reserved / padding */
            __u8    __res1;  /* reserved / padding */
            __u8    data[8] __attribute__((aligned(8)));
    };
```

In [1]:
#Import
import socket
import struct

In [2]:
# Open a socket and connect to it
sock = socket.socket(socket.PF_CAN, socket.SOCK_DGRAM, socket.CAN_BCM)

#Change this interface to match your desired connection
interface = "vcan0"

#Connect to the interface
sock.connect((interface,))

# To match the socketCAN BCM header data structure, the following struct format can be used:
# Interval seconds and useconds are platform dependent, others are 'uint32'. 
# Pad bytes are required at the end. This padding is not documented very well.
bcm_frame_format = "@3I4l2I0q"

# To match the socketCAN CAN frame structure, the following struct format can be used:
can_frame_format = "<lB3x8s"

### Message Creation
Let's simulate a brake controller broadcasting independent wheel speeds on the J1939 network. Based on the SAE J1939 Digital Annex, we have the following PGN and SPNs:


| PGN | PG Label | PG Acronym | EDP | DP | PF | PS | Multipacket | Transmission Rate | PG Data Length | Default Priority |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| 65134 | High Resolution Wheel Speed | HRW | 0 | 0 | 254 | 110 | No | 20 ms | 8 | 2 |

| SP Position in PG | SPN | SP Label | SP Description | SP Length | Resolution | Offset | Data Range |
| :-: | :-: | :-- | :-- | :-: | :-: | :-: | :-: | 
| 1-2 | 1592 | Front Axle, Left Wheel Speed | High resolution measurement of the speed of the left wheel on the front axle. | 2 bytes | 1/256 km/h per bit | 0 | 0 to 250.996 km/h |
| 3-4 | 1593 | Front Axle, Right Wheel Speed | High resolution measurement of the speed of the right wheel on the front axle. | 2 bytes | 1/256 km/h per bit | 0 | 0 to 250.996 km/h |
| 5-6 | 1594 | Rear Axle, Left Wheel Speed | High resolution measurement of the speed of the left wheel on the rear axle. | 2 bytes | 1/256 km/h per bit | 0 | 0 to 250.996 km/h |
| 7-8 | 1595 | Rear Axle, Left Wheel Speed | High resolution measurement of the speed of the left wheel on the rear axle. | 2 bytes | 1/256 km/h per bit | 0 | 0 to 250.996 km/h |

The prefered source address is 11 (0x0B) for the Brake System Controller.

In [3]:
# Since PF is > 240, we can craft the message ID for broadcast using the PDU2 format. This 
# means we assume the DA is global and the PGN uses two bytes in the ID.
# Convert these J1939 elements into an extended CAN ID.
priority = 2
pgn = 65134
sa = 11
HRW_ID = priority << 26
HRW_ID += pgn << 8
HRW_ID += sa
print("The CAN ID for HRW Broadcast is 0x{:08X}".format(HRW_ID))

The CAN ID for HRW Broadcast is 0x08FE6E0B


In [4]:
# Let's define some arbitrary SPNs
SPN_1592 = 1/256 #Front Axle, Left Wheel Speed in km/h
SPN_1593 = 100.23 #Front Axle, Right Wheel Speed in km/h
SPN_1594 = 150.8787 #Rear Axle, Left Wheel Speed in km/h
SPN_1595 = 200.213409576234 #Rear Axle, Right Wheel Speed in km/h

# Based on the resolution, we need to create an 8-byte data frame
# Apply the inverse of the resolution, then encode the integer into an 
# Intel formatted 2-byte unsigned integer.
HRW_data = struct.pack('<4H', int(SPN_1592*256),
                              int(SPN_1593*256),
                              int(SPN_1594*256),
                              int(SPN_1595*256),
                      )
print(HRW_data)

b'\x01\x00:d\xe0\x966\xc8'


In [5]:
# Now we can create a CAN Frame. This is the same as if we used CAN_RAW

#Set the extended frame format bit.
can_id = HRW_ID | socket.CAN_EFF_FLAG

#The data length code cannot be larger than 8
can_dlc = len(HRW_data)

#Pack the information into 16 bytes
can_packet = struct.pack(can_frame_format, can_id, can_dlc, HRW_data)
print(can_packet)

b'\x0bn\xfe\x88\x08\x00\x00\x00\x01\x00:d\xe0\x966\xc8'


### Create the BCM Header
The BCM header has the following elements:
* opcode
* message flag
* count
* timing intervals
* unique CAN ID for task
* number of CAN frames

The number of CAN frames is the number of frames that are attached to the overall BCM data structure. The BCM method enables not just the simple periodic single can frame, but finite bursts of multi-frame messages. We'll start simple, then work towards a more complete example.

Let's start with setting up a transmit BCM:

#### Op-Codes
From https://www.kernel.org/doc/Documentation/networking/can.txt, the following opcodes are defined:

Transmit Operations (user space to broadcast manager):

    TX_SETUP:   Create (cyclic) transmission task.

    TX_DELETE:  Remove (cyclic) transmission task, requires only can_id.

    TX_READ:    Read properties of (cyclic) transmission task for can_id.

    TX_SEND:    Send one CAN frame.

  Transmit Responses (broadcast manager to user space):

    TX_STATUS:  Reply to TX_READ request (transmission task configuration).

    TX_EXPIRED: Notification when counter finishes sending at initial interval
      'ival1'. Requires the TX_COUNTEVT flag to be set at TX_SETUP.

  
 #### Message Flags
 From https://www.kernel.org/doc/Documentation/networking/can.txt, the following flags are defined:

    SETTIMER:           Set the values of ival1, ival2 and count

    STARTTIMER:         Start the timer with the actual values of ival1, ival2
      and count. Starting the timer leads simultaneously to emit a CAN frame.

    TX_COUNTEVT:        Create the message TX_EXPIRED when count expires

    TX_ANNOUNCE:        A change of data by the process is emitted immediately.

    TX_CP_CAN_ID:       Copies the can_id from the message header to each
      subsequent frame in frames. This is intended as usage simplification. For
      TX tasks the unique can_id from the message header may differ from the
      can_id(s) stored for transmission in the subsequent struct can_frame(s).

    TX_RESET_MULTI_IDX: Reset the index for the multiple frame transmission.
    
Definitions in python are published here:
https://github.com/caran/can4python/blob/4177bcf4368fd8484ec080e2d926cb999b14c6a5/can4python/constants.py#L72


In [6]:
# BCM flags
SETTIMER = 0x0001
STARTTIMER = 0x0002
TX_COUNTEVT = 0x0004
TX_ANNOUNCE = 0x0008
TX_CP_CAN_ID = 0x0010
RX_FILTER_ID = 0x0020
RX_CHECK_DLC = 0x0040
RX_NO_AUTOTIMER = 0x0080
RX_ANNOUNCE_RESUME = 0x0100
TX_RESET_MULTI_IDX = 0x0200
RX_RTR_FRAME = 0x0400
CAN_FD_FRAME = 0x0800

### Create the BCM message and Data Structure

In [7]:
# Create the BCM Header

opcode = socket.CAN_BCM_TX_SETUP
flags = SETTIMER | STARTTIMER # These flags set and start the timers for broadcast.
count = 0 # When only one timer is needed 'count' is set to zero and only 'ival2' is used.
ival1_tv_sec  = 0 # Seconds
ival1_tv_usec = 0 # Microseconds
ival2_tv_sec  = 0  # Seconds
ival2_tv_usec = 20000 # Microseconds (20 ms per J1939)
can_id = HRW_ID
nframes = 1 # Append a single CAN frame to the end the setup.

bcm_frame = struct.pack(bcm_frame_format,
                        opcode,
                        flags,
                        count,
                        ival1_tv_sec,
                        ival1_tv_usec,
                        ival2_tv_sec,
                        ival2_tv_usec,
                        can_id,
                        nframes)
print(bcm_frame)                        

b'\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 N\x00\x00\x0bn\xfe\x08\x01\x00\x00\x00\x00\x00\x00\x00'


In [8]:
#Send out the BCM Message. This will get the message going.
# The return value is the number of bytes sent to the socket.
sock.send(bcm_frame+can_packet)

56

In a terminal, you can see the output on the virtual CAN channel:
```
debian@beaglebone:~/TruckCapeProjects$ candump -tz vcan1
 (000.000000)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.020063)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.040123)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.060174)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.080227)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.100280)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.120332)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.140410)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.160465)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.180511)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.200556)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.220608)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.240657)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.260705)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.280786)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.300835)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.320887)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.340940)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
 (000.360991)  vcan1  08FE6E0B   [8]  01 00 3A 64 E0 96 36 C8
```
The `-tz` flag on the candump command shows a timestamp starting from zero in the parentheses. Notice the near 20ms timing. The data hasn't changed at all, because we haven't told it to change yet. 

### Update the Signals and SPNs
Let's model a truck with Ackerman steering turning right. The truck will start from a stop and accelerate to a constant speed with the steering wheel turned all the way to the right. This is a low speed manuever and we'll assume no tire slip. All four wheels are turning around a common pivot point. A vector starting at the central pivot point describes the position to each wheel. We'll make a simplifing assumption that the angular velocity of all the wheel position vectors are the same.

We'll give the truck the following made-up dimensions (in meters):

Wheelbase = 6.0 

Track Width = 4.0 

Rolling radius = 1.5

Distances from the Pivot:

$ LR = 10.0 $, $ RR = 6.0 $, $ LF = 11.6 $, $ RF = 8.4 $, $ COM = 8.8 $, $ \delta_R = 45 deg $, and $ \delta_L = 32 deg $

![AckermanGeometry.png](AckermanGeometry.png)

Assume all vectors are constant length (i.e. circular motion) and the piviot is fixed.

The speed at the end of any vector is 
$$ v_i = \omega r_i $$
where $\omega$ is the angular velocity of the rotating vector, $r_i$ is the radius (length) of the vector and $v_i$ is the speed. We can calculate the angular velocity based on the speed of the center of mass and the position of the center of mass:

$$ \omega = \frac{v_{COM}}{r_{COM}} $$

Once we know $\omega$, we can determine the angular velocities of the other wheels.

$$v_i = \omega r_i$$

Note: on real systems, we use sensors at each wheel to measure actual speed. For this exercise, we are simulating a very simple case. 

Let's translate this into a Python function:

In [9]:
# Produce wheel speeds based on turn radius and COM speed
def wheel_speeds(com_speed,radii):
    #omega is in radians per second
    #radii is in meters
    #speed is in km/h
    #convert m/s to km/h by multiplying by 3.6. Why? 3600 seconds/hour divided by 1000 meters/km
    omega = (com_speed / 3.6)  / radii['COM']
    return(3.6*omega*radii['LF'],
           3.6*omega*radii['RF'],
           3.6*omega*radii['LR'],
           3.6*omega*radii['RR'])
# These align with the SPNs for the HRW PGN.

In [10]:
# If the steering angle changed or there was tire slip, these would change.
# Assume constant for now.
radii = {'COM': 8.8,
         'LF': 11.6,
         'RF': 8.4,
         'LR': 10.0,
         'RR': 6.0,
        }
com_speed = 5
wheel_speeds(com_speed,radii)

(6.59090909090909, 4.7727272727272725, 5.681818181818182, 3.4090909090909087)

In [11]:
# Write a function to take four wheel speeds and create the CAN message:
def can_HRW_speeds(com_speed):
    LF,RF,LR,RR = wheel_speeds(com_speed,radii)
    HRW_data = struct.pack('<4H', int(LF*256),
                                  int(RF*256),
                                  int(LR*256),
                                  int(RR*256))
    #Set the extended frame format bit.
    can_id = HRW_ID | socket.CAN_EFF_FLAG

    #The data length code cannot be larger than 8
    can_dlc = len(HRW_data)

    #Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, HRW_data)
    return can_packet
print(can_HRW_speeds(10))

b'\x0bn\xfe\x88\x08\x00\x00\x00.\r\x8b\t]\x0b\xd1\x06'


In [12]:
# Define the BCM header to update new values only
# We are not setting timers, just updating values:
# Create the BCM Header

opcode = socket.CAN_BCM_TX_SETUP
flags = 0 # Default zero just updates the message
count = 0 # no timers
ival1_tv_sec  = 0 # Seconds
ival1_tv_usec = 0 # Microseconds
ival2_tv_sec  = 0  # Seconds
ival2_tv_usec = 0
can_id = HRW_ID
nframes = 1 # Append a single CAN frame to the end the setup.

bcm_frame = struct.pack(bcm_frame_format,
                        opcode,
                        flags,
                        count,
                        ival1_tv_sec,
                        ival1_tv_usec,
                        ival2_tv_sec,
                        ival2_tv_usec,
                        can_id,
                        nframes)
print(bcm_frame)       

b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0bn\xfe\x08\x01\x00\x00\x00\x00\x00\x00\x00'


In [13]:
#Set everything to zeros:
can_packet = can_HRW_speeds(0)
sock.send(bcm_frame+can_packet)

56

In [14]:
# Let's conduct a realtime simulation for 3 seconds
# Turn on candump to see the live data 
import time
deltaT = 0.010 #seconds
initial_velocity = 0
acceleration = 0.1 * 9.81 * 3.6 #m/s to km/h/s
v = initial_velocity 
t = 0
for i in range(300):
    # Send out the data
    print("{:0.2f},{:0.4f}".format(t,v))
    can_packet = can_HRW_speeds(v)
    sock.send(bcm_frame+can_packet)
    # update to the next step (Euler's method)
    time.sleep(deltaT)
    t += deltaT
    v += acceleration*deltaT #update the COM speed

0.00,0.0000
0.01,0.0353
0.02,0.0706
0.03,0.1059
0.04,0.1413
0.05,0.1766
0.06,0.2119
0.07,0.2472
0.08,0.2825
0.09,0.3178
0.10,0.3532
0.11,0.3885
0.12,0.4238
0.13,0.4591
0.14,0.4944
0.15,0.5297
0.16,0.5651
0.17,0.6004
0.18,0.6357
0.19,0.6710
0.20,0.7063
0.21,0.7416
0.22,0.7770
0.23,0.8123
0.24,0.8476
0.25,0.8829
0.26,0.9182
0.27,0.9535
0.28,0.9888
0.29,1.0242
0.30,1.0595
0.31,1.0948
0.32,1.1301
0.33,1.1654
0.34,1.2007
0.35,1.2361
0.36,1.2714
0.37,1.3067
0.38,1.3420
0.39,1.3773
0.40,1.4126
0.41,1.4480
0.42,1.4833
0.43,1.5186
0.44,1.5539
0.45,1.5892
0.46,1.6245
0.47,1.6599
0.48,1.6952
0.49,1.7305
0.50,1.7658
0.51,1.8011
0.52,1.8364
0.53,1.8717
0.54,1.9071
0.55,1.9424
0.56,1.9777
0.57,2.0130
0.58,2.0483
0.59,2.0836
0.60,2.1190
0.61,2.1543
0.62,2.1896
0.63,2.2249
0.64,2.2602
0.65,2.2955
0.66,2.3309
0.67,2.3662
0.68,2.4015
0.69,2.4368
0.70,2.4721
0.71,2.5074
0.72,2.5428
0.73,2.5781
0.74,2.6134
0.75,2.6487
0.76,2.6840
0.77,2.7193
0.78,2.7546
0.79,2.7900
0.80,2.8253
0.81,2.8606
0.82,2.8959
0.83

 Part of the the `candump -tz vcan1` output is as follows
 ```
 (002.867990)  vcan1  08FE6E0B   [8]  00 00 00 00 00 00 00 00
 (002.888051)  vcan1  08FE6E0B   [8]  00 00 00 00 00 00 00 00
 (002.908148)  vcan1  08FE6E0B   [8]  17 00 11 00 14 00 0C 00
 (002.928230)  vcan1  08FE6E0B   [8]  23 00 19 00 1E 00 12 00
 (002.948293)  vcan1  08FE6E0B   [8]  3B 00 2B 00 33 00 1E 00
 (002.968368)  vcan1  08FE6E0B   [8]  53 00 3C 00 47 00 2B 00
 (002.988471)  vcan1  08FE6E0B   [8]  6B 00 4D 00 5C 00 37 00
 (003.008571)  vcan1  08FE6E0B   [8]  83 00 5E 00 71 00 43 00
 (003.028655)  vcan1  08FE6E0B   [8]  8F 00 67 00 7B 00 49 00
 (003.048767)  vcan1  08FE6E0B   [8]  A6 00 78 00 8F 00 56 00
 (003.068836)  vcan1  08FE6E0B   [8]  BE 00 8A 00 A4 00 62 00
 (003.088932)  vcan1  08FE6E0B   [8]  D6 00 9B 00 B8 00 6E 00
 (003.109049)  vcan1  08FE6E0B   [8]  E2 00 A3 00 C3 00 75 00
 (003.129145)  vcan1  08FE6E0B   [8]  FA 00 B5 00 D7 00 81 00
 (003.149317)  vcan1  08FE6E0B   [8]  12 01 C6 00 EC 00 8D 00
 (003.169388)  vcan1  08FE6E0B   [8]  29 01 D7 00 00 01 9A 00
 (003.189463)  vcan1  08FE6E0B   [8]  35 01 E0 00 0B 01 A0 00
 (003.209535)  vcan1  08FE6E0B   [8]  4D 01 F1 00 1F 01 AC 00
 (003.229644)  vcan1  08FE6E0B   [8]  65 01 02 01 34 01 B8 00
 (003.249730)  vcan1  08FE6E0B   [8]  7D 01 14 01 48 01 C5 00
 (003.269813)  vcan1  08FE6E0B   [8]  95 01 25 01 5D 01 D1 00
 (003.289913)  vcan1  08FE6E0B   [8]  A1 01 2E 01 67 01 D7 00
 (003.310000)  vcan1  08FE6E0B   [8]  B8 01 3F 01 7C 01 E4 00
 (003.330094)  vcan1  08FE6E0B   [8]  D0 01 50 01 90 01 F0 00
 (003.350170)  vcan1  08FE6E0B   [8]  DC 01 59 01 9A 01 F6 00
 (003.370247)  vcan1  08FE6E0B   [8]  F4 01 6A 01 AF 01 02 01
 (003.390328)  vcan1  08FE6E0B   [8]  0C 02 7B 01 C4 01 0F 01
 (003.410442)  vcan1  08FE6E0B   [8]  24 02 8C 01 D8 01 1B 01
 (003.430532)  vcan1  08FE6E0B   [8]  3C 02 9E 01 ED 01 27 01
 (003.450622)  vcan1  08FE6E0B   [8]  47 02 A6 01 F7 01 2E 01
 (003.470735)  vcan1  08FE6E0B   [8]  5F 02 B8 01 0B 02 3A 01
 (003.490843)  vcan1  08FE6E0B   [8]  77 02 C9 01 20 02 46 01
 (003.510952)  vcan1  08FE6E0B   [8]  8F 02 DA 01 35 02 53 01
 (003.531065)  vcan1  08FE6E0B   [8]  9B 02 E3 01 3F 02 59 01
 (003.551155)  vcan1  08FE6E0B   [8]  B3 02 F4 01 53 02 65 01
 ...
```
Since this is a right turn, the left wheels need to spin faster. You'll notice the second and 6th byte corresond to the km/h values of the left side and they increase speed faster than the right wheels.

Notice the timing on the vcan is still 20ms, even though the simulation ran at 10ms. The CAN broadcasts what it was last updated with when the timer says it's ready to broadcast. There can be many updates in between each broadcast. If no update is received, then the message is repeated. 

Try to run the simulation with updates every 90 ms. What does the candump timing record show? Do messages repeat as you would predict?

Once the simulation is finished, the CAN messages keep playing the last known value:
```
 (014.081530)  vcan1  08FE6E0B   [8]  0A 2E 57 21 B0 27 D0 17
 (014.101595)  vcan1  08FE6E0B   [8]  22 2E 68 21 C5 27 DC 17
 (014.121702)  vcan1  08FE6E0B   [8]  3A 2E 79 21 D9 27 E9 17
 (014.141777)  vcan1  08FE6E0B   [8]  51 2E 8A 21 EE 27 F5 17
 (014.161862)  vcan1  08FE6E0B   [8]  69 2E 9C 21 02 28 01 18
 (014.181933)  vcan1  08FE6E0B   [8]  75 2E A4 21 0D 28 07 18
 (014.202024)  vcan1  08FE6E0B   [8]  81 2E AD 21 17 28 0E 18
 (014.222123)  vcan1  08FE6E0B   [8]  81 2E AD 21 17 28 0E 18
 (014.242230)  vcan1  08FE6E0B   [8]  81 2E AD 21 17 28 0E 18
 (014.262319)  vcan1  08FE6E0B   [8]  81 2E AD 21 17 28 0E 18
 (014.282411)  vcan1  08FE6E0B   [8]  81 2E AD 21 17 28 0E 18
```


Try to run the simulation with updates every 90 ms. What does the candump timing record show? Do messages repeat as you would predict?
```
 (024.729463)  vcan1  08FE6E0B   [8]  00 00 00 00 00 00 00 00
 (024.749541)  vcan1  08FE6E0B   [8]  00 00 00 00 00 00 00 00
 (024.769612)  vcan1  08FE6E0B   [8]  6B 00 4D 00 5C 00 37 00
 (024.789690)  vcan1  08FE6E0B   [8]  6B 00 4D 00 5C 00 37 00
 (024.809738)  vcan1  08FE6E0B   [8]  6B 00 4D 00 5C 00 37 00
 (024.829780)  vcan1  08FE6E0B   [8]  6B 00 4D 00 5C 00 37 00
 (024.849844)  vcan1  08FE6E0B   [8]  D6 00 9B 00 B8 00 6E 00
 (024.870044)  vcan1  08FE6E0B   [8]  D6 00 9B 00 B8 00 6E 00
 (024.890176)  vcan1  08FE6E0B   [8]  D6 00 9B 00 B8 00 6E 00
 (024.910234)  vcan1  08FE6E0B   [8]  D6 00 9B 00 B8 00 6E 00
 (024.930279)  vcan1  08FE6E0B   [8]  D6 00 9B 00 B8 00 6E 00
 (024.950345)  vcan1  08FE6E0B   [8]  41 01 E9 00 15 01 A6 00
 (024.970392)  vcan1  08FE6E0B   [8]  41 01 E9 00 15 01 A6 00
 (024.990445)  vcan1  08FE6E0B   [8]  41 01 E9 00 15 01 A6 00
 (025.010496)  vcan1  08FE6E0B   [8]  41 01 E9 00 15 01 A6 00
 (025.030572)  vcan1  08FE6E0B   [8]  AD 01 36 01 71 01 DD 00
```
Since the simulation time step is slower than the CAN update rate, the message gets repeated until there is new data. This happens every 4 or 5 messages when the update rate is 90ms for the simulation and 20ms for the CAN. 

### Send multiframe message
Let's send a VIN according to J1939 every 5 seconds 3 times

The VIN will be encoded as 17 ascii characters and sent using the J1939 transport protocol.
There will be 4 messages:

1. Connection management
2. Data 1 
3. Data 2
4. Data 3

The VIN PGN is 65260 (0xFEEC). We'll usa a dynamically allocated address (0xA0) for the source and a priority of 6. With this information, we'll use the ID of 0x18FEECA0 for the BCM.

In [15]:
opcode = socket.CAN_BCM_TX_SETUP
flags = SETTIMER | STARTTIMER | TX_COUNTEVT # These flags set and start the timers for broadcast.
count = 4 # When only one timer is needed 'count' is set to zero and only 'ival2' is used.
ival1_tv_sec  = 0 # Seconds
ival1_tv_usec = 50000 # Microseconds
ival2_tv_sec  = 0 # Seconds
ival2_tv_usec = 0 # Set this to zero to stop after the initial count.
can_id = 0x18FEECA0
nframes = 4 # Append a sequence of BAM messages

bcm_frame = struct.pack(bcm_frame_format,
                        opcode,
                        flags,
                        count,
                        ival1_tv_sec,
                        ival1_tv_usec,
                        ival2_tv_sec,
                        ival2_tv_usec,
                        can_id,
                        nframes)
print(bcm_frame)                        

b'\x01\x00\x00\x00\x07\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00P\xc3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0\xec\xfe\x18\x04\x00\x00\x00\x00\x00\x00\x00'


In [16]:
#create a function to send VIN
#Build the CAN Packets for the VIN BAM message:
def sendVIN():
    opcode = socket.CAN_BCM_TX_SETUP
    flags = SETTIMER | STARTTIMER | TX_COUNTEVT # These flags set and start the timers for broadcast.
    count = 4 # When only one timer is needed 'count' is set to zero and only 'ival2' is used.
    ival1_tv_sec  = 0 # Seconds
    ival1_tv_usec = 50000 # Microseconds
    ival2_tv_sec  = 0 # Seconds
    ival2_tv_usec = 0 # Set this to zero to stop after the initial count.
    can_id = 0x18FEECA0
    nframes = 4 # Append a sequence of BAM messages

    bcm_frame = struct.pack(bcm_frame_format,
                            opcode,
                            flags,
                            count,
                            ival1_tv_sec,
                            ival1_tv_usec,
                            ival2_tv_sec,
                            ival2_tv_usec,
                            can_id,
                            nframes)

    #Connection managment
    can_id = 0x18ECFFA0 | socket.CAN_EFF_FLAG

    connection_data = struct.pack('<BHBBBBB',32,18,4,255,0xEC,0xFE,0x00)

    can_dlc = len(connection_data)

    #Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, connection_data)

    #Data Transport
    can_id = 0x18EBFFA0 | socket.CAN_EFF_FLAG

    transport_data = b'\x01THISIsA'
    can_packet += struct.pack(can_frame_format, can_id, can_dlc, transport_data)

    transport_data = b'\x02nExampl'
    can_packet += struct.pack(can_frame_format, can_id, can_dlc, transport_data)

    transport_data = b'\x03eVIN\xFF\xFF\xFF'
    can_packet += struct.pack(can_frame_format, can_id, can_dlc, transport_data)
    return (sock.send(bcm_frame+can_packet))

In [17]:
#Run this to start the sequence. Monitor with candump.
sendVIN()

104

In [18]:
# Rerun to send the same burst output
sendVIN()                  

104

The candump output gives these two bursts. Each burst has messages that are 50ms apart.
```
 (389.090370)  vcan1  18ECFFA0   [8]  20 12 00 04 FF EC FE 00
 (389.148373)  vcan1  18EBFFA0   [8]  01 54 68 69 73 49 73 41
 (389.198478)  vcan1  18EBFFA0   [8]  02 6E 45 78 61 6D 70 6C
 (389.248554)  vcan1  18EBFFA0   [8]  03 65 56 49 4E FF FF FF
 
 (391.803641)  vcan1  18ECFFA0   [8]  20 12 00 04 FF EC FE 00
 (391.854598)  vcan1  18EBFFA0   [8]  01 54 68 69 73 49 73 41
 (391.904681)  vcan1  18EBFFA0   [8]  02 6E 45 78 61 6D 70 6C
 (391.954756)  vcan1  18EBFFA0   [8]  03 65 56 49 4E FF FF FF
```

Doing this with a service tool attached shows the message was transmitted.

![DGVINExample.png](DGVINExample.png)

## Receiving Messages
We can alus use the BCM approach to look for requests. 

A request message has a PGN of 0xEA00.

```
 ival1: Send RX_TIMEOUT when a received message is not received again within
    the given time. When START_TIMER is set at RX_SETUP the timeout detection
    is activated directly - even without a former CAN frame reception.

  ival2: Throttle the received message rate down to the value of ival2. This
    is useful to reduce messages for the application when the signal inside the
    CAN frame is stateless as state changes within the ival2 periode may get
    lost.
```

In [19]:
#Look for request messages: PGN = 0xEA00 every 200 ms.
opcode = socket.CAN_BCM_RX_SETUP
flags = RX_FILTER_ID
count= 0 
ival1_tv_sec=0
ival1_tv_usec=0
ival2_tv_sec=0
ival2_tv_usec=0
can_id = 0x98EAFFF9 #An identifier for this filter
nframes=0
bcm_frame = struct.pack(bcm_frame_format,
                        opcode,
                        flags,
                        count,
                        ival1_tv_sec,
                        ival1_tv_usec,
                        ival2_tv_sec,
                        ival2_tv_usec,
                        can_id,
                        nframes)
sock.send(bcm_frame)

40

In [20]:
print(sock.recv(56))

b'\t\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00P\xc3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0\xec\xfe\x18\x00\x00\x00\x00\x10\x00\x00\x00'


### Build a routine to respond to request messages
These are borrowed from [03_Parse J1939 Single Frame Messages.ipynb](03_Parse%20J1939%20Single%20Frame%20Messages.ipynb) 


In [21]:
def unpack_CAN(can_packet,display=False):
    can_id, can_dlc, can_data = struct.unpack(can_frame_format, can_packet)
    extended_frame = bool(can_id & socket.CAN_EFF_FLAG)
    if extended_frame:
        can_id &= socket.CAN_EFF_MASK
        can_id_string = "{:08X}".format(can_id)
    else: #Standard Frame
        can_id &= socket.CAN_SFF_MASK
        can_id_string = "{:03X}".format(can_id)
    if display:
        hex_data_string = ' '.join(["{:02X}".format(b) for b in can_data[:can_dlc]])
        print("{} {} [{}] {}".format(interface, can_id_string, can_dlc, hex_data_string))
    return can_id, can_dlc, can_data

In [22]:
#parse J1939 protocol data unit information from the ID using bit masks and shifts
PRIORITY_MASK = 0x1C000000
EDP_MASK      = 0x02000000
DP_MASK       = 0x01000000
PF_MASK       = 0x00FF0000
PS_MASK       = 0x0000FF00
SA_MASK       = 0x000000FF
PDU1_PGN_MASK = 0x03FF0000
PDU2_PGN_MASK = 0x03FFFF00

def get_j1939_from_id(can_id):
    #priority
    priority = (PRIORITY_MASK & can_id) >> 26

    #Extended Data Page
    edp = (EDP_MASK & can_id) >> 25
    
    # Data Page
    dp = (DP_MASK & can_id) >> 24
    
    # Protocol Data Unit (PDU) Format
    PF = (can_id & PF_MASK) >> 16
    
    # Protocol Data Unit (PDU) Specific
    PS = (can_id & PS_MASK) >> 8
    
    # Determine the Parameter Group Number and Destination Address
    if PF >= 0xF0: #240

        # PDU 2 format, include the PS as a group extension
        DA = 255
        PGN = (can_id & PDU2_PGN_MASK) >> 8
    else:
        PGN = (can_id & PDU1_PGN_MASK) >> 8
        DA = PS
    # Source address
    SA = (can_id & SA_MASK)
    
    return priority,PGN,DA,SA

In [23]:
# Press Esc , i, i to interrupt the Kernel
for i in range(500):
    can_bcm = sock.recv(56)
    can_packet = can_bcm[-16:]
    #Parse the bytes into a CAN message
    can_id, can_dlc, can_data = unpack_CAN(can_packet)
    #Parse the CAN ID into J1939
    priority,pgn,da,sa = get_j1939_from_id(can_id)
    print(priority,pgn,da,sa,can_data)
    # only global requests from the service tool are processed
    requested_pgn = struct.unpack("<L",can_data[0:4])[0]
    #print("{:05X}".format(requested_pgn))
    if requested_pgn == 0xFEEC: #VIN
        for i in range(3): #Sending multiple times increases chances for reception
            sendVIN()
            time.sleep(.3)
        break

0 0 0 0 b'\x00\x00\x00\x00H\xde\x1a\xc0'


KeyboardInterrupt: 

If we have a diagnostic application, like DG Diagnostics, running, the VIN should have been requested and the BCM would have responded with 3 bursts. The program does not have to filter all the messages to look for the Request Message. 

## Delete the BCM messages
When we are all finish, you can delete the Broadcast Message

In [24]:
#Delete the VIN Burst session
opcode = socket.CAN_BCM_TX_DELETE
flags = 0
count= 0 
ival1_tv_sec=0
ival1_tv_usec=0
ival2_tv_sec=0
ival2_tv_usec=0
can_id = 0x18FEECA0
nframes=0
bcm_frame = struct.pack(bcm_frame_format,
                        opcode,
                        flags,
                        count,
                        ival1_tv_sec,
                        ival1_tv_usec,
                        ival2_tv_sec,
                        ival2_tv_usec,
                        can_id,
                        nframes)
# Send this only once. 
sock.send(bcm_frame)

40

In [25]:
# Delete a CAN Stream for the brake message
opcode = socket.CAN_BCM_TX_DELETE
flags = 0
count= 0 
ival1_tv_sec=0
ival1_tv_usec=0
ival2_tv_sec=0
ival2_tv_usec=0
can_id = HRW_ID
nframes=0
bcm_frame = struct.pack(bcm_frame_format,
                        opcode,
                        flags,
                        count,
                        ival1_tv_sec,
                        ival1_tv_usec,
                        ival2_tv_sec,
                        ival2_tv_usec,
                        can_id,
                        nframes)
print(bcm_frame)                        

sock.send(bcm_frame)

b'\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0bn\xfe\x08\x00\x00\x00\x00\x00\x00\x00\x00'


40

### Summary
In this notebook we introduced transmitting and receiving with the SocketCAN Broacast Manager. There were examples of asynchronously updating the broadcast CAN signals based on a real-time simulation. A message burst was created and sent with the BCM. 

Using the BCM alleviates the need for a program to maintain tight time requirements and relegate that operation to the SocketCAN kernel module. 